In [1]:
import pandas as pd
import numpy as np
from pathlib import Path

In [ ]:
basic_directory = Path.cwd()                            # directory of the file - folder "code"
data_directory = basic_directory.parent / "data"        # go one level up and choose folder "data"

# It's a common convention to add a _df suffix to a variable name to indicate it's a DataFrame.
flights_df  = pd.read_csv(data_directory / "flights.csv")
airlines_df = pd.read_csv(data_directory / "airlines.csv")
airports_df = pd.read_csv(data_directory / "airports.csv")
planes_df   = pd.read_csv(data_directory / "planes.csv")
weather_df  = pd.read_csv(data_directory / "weather.csv")

# Convert to datetime
flights_df['time_hour'] = pd.to_datetime(flights_df['time_hour'])
weather_df['time_hour'] = pd.to_datetime(weather_df['time_hour'])

# Remove timezone
weather_df['time_hour'] = weather_df['time_hour'].dt.tz_localize(None)

'''
Convert the 'time_hour' column in flights_df and weather_df to datetime64[ns] type, as it was 'object'.
This was necessary because a merge on date/time types was not possible otherwise.

The dtype for flights_df was datetime64[ns], while for weather_df it was datetime64[ns, UTC]. Removed the UTC timezone.
'''

'\nПреобразуем колонку time_hour в flights_df и weather_df в тип datetime64[ns], потому что был object\nПотому что нельзя было делать merge по дате/времени\n\nУ flights_df был тип - datetime64[ns]. У weather_df - datetime64[ns, UTC]. Убрал таймзону UTC\n'

In [9]:
# airlines с CASE
airlines_part = (
    airlines_df[['name']]
    .head(5)
    .assign(
        entity_type=lambda df: np.where(
            df['name'].str[0].str.upper().isin(['A','E','I','O','U']),
            'airlines_vowel',
            'airlines_consonant'
        )
    )
)

airports_part = (
    airports_df[['name']]
    .head(5)
    .assign(entity_type='airports')
)

planes_part = (
    planes_df[['manufacturer']]
    .head(5)
    .rename(columns={'manufacturer':'name'}) # переименовать manufacturer в name, чтобы не создалась отдельная колонка
    .assign(entity_type='manufacturer')
)

union_all = pd.concat([airlines_part, airports_part, planes_part], ignore_index=True)
print(union_all)

'''
SELECT
    CASE
        WHEN UPPER(LEFT(name, 1)) IN ('A', 'E', 'I', 'O', 'U')
        THEN 'airlines_vowel'
        ELSE 'airlines_consonant'   
    END AS entity_type, name
FROM airlines
UNION ALL

SELECT 'airports', name -- для последующий SELECT в своей колонке не нужно указывать название
FROM airports
UNION ALL

SELECT 'manufacturer', manufacturer
FROM planes
WHERE manufacturer IS NOT NULL; -- Исключаем пустые значения
'''

                             name         entity_type
0               Endeavor Air Inc.      airlines_vowel
1          American Airlines Inc.      airlines_vowel
2            Alaska Airlines Inc.      airlines_vowel
3                 JetBlue Airways  airlines_consonant
4            Delta Air Lines Inc.  airlines_consonant
5               Lansdowne Airport            airports
6   Moton Field Municipal Airport            airports
7             Schaumburg Regional            airports
8                 Randall Airport            airports
9           Jekyll Island Airport            airports
10                        EMBRAER        manufacturer
11               AIRBUS INDUSTRIE        manufacturer
12               AIRBUS INDUSTRIE        manufacturer
13               AIRBUS INDUSTRIE        manufacturer
14                        EMBRAER        manufacturer


"\nSELECT\n    CASE\n        WHEN UPPER(LEFT(name, 1)) IN ('A', 'E', 'I', 'O', 'U')\n        THEN 'airlines_vowel'\n        ELSE 'airlines_consonant'   \n    END AS entity_type, name\nFROM airlines\nUNION ALL\n\nSELECT 'airports', name -- для последующий SELECT в своей колонке не нужно указывать название\nFROM airports\nUNION ALL\n\nSELECT 'manufacturer', manufacturer\nFROM planes\nWHERE manufacturer IS NOT NULL; -- Исключаем пустые значения\n"

In [ ]:
intersect = flights_df.loc[flights_df["origin"]=="JFK", "carrier"] \
    .isin(flights_df.loc[flights_df["origin"]=="LGA", "carrier"]) \
    & flights_df.loc[flights_df["origin"]=="JFK", "carrier"] \
    .isin(flights_df.loc[flights_df["origin"]=="EWR", "carrier"])

result = flights_df.loc[flights_df["origin"]=="JFK", "carrier"][intersect].unique()
print(result)
'''
-- Авиакомпании, которые летают из всех трех аэропортов NYC

flights_df["origin"]=="JFK" - создаёт массив True/False для всех значений в origin. True если "JFK"
flights_df.loc[...] - берёт только те значения, где flights_df["origin"]=="JFK" выдаёт True
..., "carrier"] - выдаёт только колонку "carrier" оттуда

.isin() проверяет, содержиться ли это значение в другом списке (есть ли True в LGA там же, где и в JFK)

.unique() - для того чтобы не повторялись компании столько раз, сколько для трёх аэропортов было True. Только один
'''

'''
Аналог .isin, только с ним удобнее.

jfk = set(flights_df.loc[flights_df["origin"]=="JFK", "carrier"])
lga = set(flights_df.loc[flights_df["origin"]=="LGA", "carrier"])
ewr = set(flights_df.loc[flights_df["origin"]=="EWR", "carrier"])

# пересечение трёх множеств
intersect_carriers = jfk & lga & ewr
print(intersect_carriers)

'''

'''
SELECT carrier FROM flights WHERE origin = 'JFK'
INTERSECT
SELECT carrier FROM flights WHERE origin = 'LGA'
INTERSECT  
SELECT carrier FROM flights WHERE origin = 'EWR';
'''

['B6' 'AA' 'UA' 'DL' '9E' 'US' 'MQ' 'EV']


"\nSELECT carrier FROM flights WHERE origin = 'JFK'\nINTERSECT\nSELECT carrier FROM flights WHERE origin = 'LGA'\nINTERSECT  \nSELECT carrier FROM flights WHERE origin = 'EWR';\n"

In [ ]:
aa_dests = set(flights_df.loc[flights_df['carrier']=="AA", "dest"])
dl_dests = set(flights_df.loc[flights_df['carrier']=="DL", "dest"])

intersect = aa_dests & dl_dests
print(intersect)
'''
если делать через .isin(), то сначала строится булевый массив (True/False для каждой строки),
потом берутся True и сравниваются с другим массивом. Это долго, поэтому лучше через set() & set()

set() & set() - сразу пересечение как в INTERSECT (SQL)
'''

'''
-- Куда летали и American Airlines, и Delta
SELECT dest 
FROM flights
WHERE carrier = 'AA'
INTERSECT
SELECT dest 
FROM flights 
WHERE carrier = 'DL';
'''

{'TPA', 'SJU', 'AUS', 'SEA', 'BOS', 'SAN', 'LAX', 'SFO', 'LAS', 'PBI', 'MIA', 'STT', 'MCO', 'FLL'}


"\n-- Куда летали и American Airlines, и Delta\nSELECT dest \nFROM flights\nWHERE carrier = 'AA'\nINTERSECT\nSELECT dest \nFROM flights \nWHERE carrier = 'DL';\n"

In [21]:
#EXCEPT
# В какие аэропорты прилетают, но из которых НЕ вылетают?
dest_set = set(flights_df['dest'])
origin_set = set(flights_df['origin'])

result = dest_set - origin_set
print(result)

'''
SELECT dest
FROM flights
EXCEPT
SELECT origin
FROM flights;
'''

{'GSO', 'MEM', 'PHX', 'TPA', 'XNA', 'MDW', 'SDF', 'RDU', 'LGB', 'DCA', 'GSP', 'CAK', 'SMF', 'PHL', 'BNA', 'SAT', 'RSW', 'MCI', 'IND', 'PIT', 'DFW', 'LAS', 'PVD', 'SAV', 'CRW', 'OAK', 'STT', 'ABQ', 'CMH', 'GRR', 'DEN', 'ANC', 'BGR', 'BHM', 'PWM', 'CHO', 'ACK', 'MKE', 'MIA', 'PSE', 'IAH', 'IAD', 'TYS', 'STL', 'LAX', 'SFO', 'AVL', 'BTV', 'SRQ', 'SLC', 'BZN', 'SJU', 'MHT', 'SEA', 'BDL', 'MYR', 'AUS', 'BOS', 'ALB', 'TUL', 'OKC', 'TVC', 'JAC', 'FLL', 'MCO', 'BUF', 'BUR', 'HOU', 'MVY', 'SAN', 'PDX', 'ATL', 'CVG', 'OMA', 'MSN', 'ORD', 'BWI', 'EGE', 'CLT', 'MSP', 'DAY', 'DSM', 'ORF', 'JAX', 'CHS', 'MSY', 'HNL', 'CAE', 'EYW', 'CLE', 'RIC', 'BQN', 'ILM', 'SNA', 'DTW', 'ROC', 'PBI', 'SJC', 'SYR'}


'\nSELECT dest\nFROM flights\nEXCEPT\nSELECT origin\nFROM flights;\n'

In [20]:
# Из каких каких аэропортов вылетают, но в которые НЕ прилетают?

dest_set = set(flights_df['dest'])
origin_set = set(flights_df['origin'])

result = origin_set - dest_set
print(result)


{'LGA', 'EWR', 'JFK'}


In [ ]:
#Самолеты, которые есть в регистре, но никогда не летали
pl_tail = set(planes_df['tailnum'])
fl_tail = set(flights_df['tailnum'].dropna())

result = pl_tail - fl_tail

print(len(result), "самолётов ни разу не летали")
print(result)
'''
SELECT tailnum 
FROM planes
EXCEPT
SELECT tailnum 
FROM flights 
WHERE tailnum IS NOT NULL; -- 998 самолётов ни разу не летали
'''

998 самолётов ни разу не летали
{'N563AS', 'N666UA', 'N959DN', 'N254WN', 'N449US', 'N911DA', 'N516LR', 'N126UW', 'N523SW', 'N828MH', 'N252WN', 'N561UW', 'N8605E', 'N184AT', 'N383AA', 'N659UA', 'N8620H', 'N672UA', 'N75432', 'N390SW', 'N685SW', 'N601WN', 'N768SK', 'N8896A', 'N284AT', 'N338AT', 'N627AW', 'N269WN', 'N698DL', 'N636WN', 'N643DL', 'N944AT', 'N910DN', 'N435AS', 'N568AS', 'N398SW', 'N557AS', 'N676UA', 'N6710E', 'N797SK', 'N8618N', 'N534AS', 'N943AT', 'N552AA', 'N103US', 'N531US', 'N957DN', 'N910FJ', 'N978SW', 'N658UA', 'N658DL', 'N905DA', 'N805AY', 'N826AW', 'N318AS', 'N444US', 'N870AS', 'N529VA', 'N641SW', 'N480AA', 'N609SW', 'N545UW', 'N901WN', 'N392SW', 'N556NW', 'N31131', 'N710SK', 'N555AY', 'N3755D', 'N68807', 'N432US', 'N521LR', 'N654SW', 'N972AT', 'N440US', 'N398AA', 'N485WN', 'N787SA', 'N619SW', 'N738CB', 'N764SW', 'N27205', 'N902FJ', 'N671UA', 'N835MH', 'N8317M', 'N518LR', 'N830AW', 'N683BR', 'N371SW', 'N7726A', 'N975AT', 'N858AS', 'N660UA', 'N345NW', 'N378SW', 'N701SK

'\nSELECT tailnum \nFROM planes\nEXCEPT\nSELECT tailnum \nFROM flights \nWHERE tailnum IS NOT NULL; -- 998 самолётов ни разу не летали\n'